In [6]:
import os
import glob
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [7]:
algoritmos = [
    "ACO",
    "ES_SA",
    "ES_VND",
]

In [8]:
instancias = [
    "Bur26a",
    "Chr12a",
    "Els19",
    "Esc16a",
    "Esc32a",
    "Had12",
    "Kra30a",
    "Lipa20a",
    "Tai256c",
    "Wil50"
]


In [9]:
pasta = './'

In [ ]:
import os
import glob
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# Usa a pasta atual (onde estão os CSVs)
pasta = "."

# Busca todos os arquivos CSV com o padrão iteracoes_*
arquivos = glob.glob(os.path.join(pasta, "iteracoes_*.csv"))

# Dicionário: instancia -> {algoritmo: dataframe processado}
dados = {}

# Tempos alvo (0 até 600 segundos, a cada 120)
tempos_alvo = np.arange(0, 601, 120)

for arquivo in arquivos:
    nome = os.path.basename(arquivo)
    partes = nome.replace(".csv", "").split("_")
    instancia = partes[1]
    algoritmo = "_".join(partes[2:])  # Ex: ES_SA, ACO, ES_VND

    # Lê CSV
    df = pd.read_csv(arquivo)

    # Detecta nome correto da coluna de tempo
    if "tempo_decorrido" in df.columns:
        tempo_col = "tempo_decorrido"
    elif "tempo_decorrido_s" in df.columns:
        tempo_col = "tempo_decorrido_s"
    else:
        raise KeyError(f"Coluna de tempo não encontrada em {arquivo}")

    # Ordena por tempo
    df = df.sort_values(tempo_col)

    # Lista de amostrados
    amostrados = []
    for t in tempos_alvo:
        # Pega o índice do tempo mais próximo
        idx = (df[tempo_col] - t).abs().idxmin()
        valor = df.loc[idx, "melhor_custo"]
        amostrados.append(valor)

    # Se terminar antes dos 600s, repete último valor
    while len(amostrados) < len(tempos_alvo):
        amostrados.append(amostrados[-1])

    df_amostrado = pd.DataFrame({
        "tempo_decorrido": tempos_alvo,
        "melhor_custo": amostrados
    })

    # Armazena no dicionário
    if instancia not in dados:
        dados[instancia] = {}
    dados[instancia][algoritmo] = df_amostrado

# Gera um gráfico por instância (com 3 algoritmos por gráfico)
for instancia, algoritmos in dados.items():
    plt.figure(figsize=(8, 5))
    for algoritmo, df in algoritmos.items():
        plt.plot(df["tempo_decorrido"], df["melhor_custo"], marker="o", label=algoritmo)
    plt.title(f"Evolução da Solução - {instancia}")
    plt.xlabel("Tempo decorrido (s)")
    plt.ylabel("Melhor custo")
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.savefig(f"evolucao_{instancia}.png", dpi=300)
    plt.close()


KeyError: 'tempo_decorrido'